In [44]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

# Load Dataset

In [3]:
data_path = '../data/jutsus.jsonl'
df = pd.read_json(data_path, lines=True)
df.head()

jutsu_name  \
0  Adamantine Sealing Chains: Spiral Formation   
1                      Adamantine Power: Acala   
2                       Adamantine Prison Wall   
3     Adamantine Seal: Monkey Yang Suppression   
4                                      Acrobat   

                                          jutsu_type  \
0  Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...   
1                  Kekkei Genkai, Ninjutsu, Taijutsu   
2              Ninjutsu, Clone Techniques, Bukijutsu   
3          Ninjutsu, Fūinjutsu, Cooperation Ninjutsu   
4                                 Taijutsu, Kenjutsu   

                                   jutsu_description  
0  Kushina uses her chains to form a barrier whil...  
1  Hashirama kicks the opponent away and raises s...  
2  After using Transformation: Adamantine Staff, ...  
3  After placing fūinjutsu tags in an area, the u...  
4  The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...

In [4]:
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Taijutsu" in jutsu:
        return "Taijutsu"

In [5]:
df['jutsu_type_simplified'] = df['jutsu_type'].apply(simplify_jutsu)

In [6]:
df.head()

jutsu_name  \
0  Adamantine Sealing Chains: Spiral Formation   
1                      Adamantine Power: Acala   
2                       Adamantine Prison Wall   
3     Adamantine Seal: Monkey Yang Suppression   
4                                      Acrobat   

                                          jutsu_type  \
0  Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...   
1                  Kekkei Genkai, Ninjutsu, Taijutsu   
2              Ninjutsu, Clone Techniques, Bukijutsu   
3          Ninjutsu, Fūinjutsu, Cooperation Ninjutsu   
4                                 Taijutsu, Kenjutsu   

                                   jutsu_description jutsu_type_simplified  
0  Kushina uses her chains to form a barrier whil...              Ninjutsu  
1  Hashirama kicks the opponent away and raises s...              Ninjutsu  
2  After using Transformation: Adamantine Staff, ...              Ninjutsu  
3  After placing fūinjutsu tags in an area, the u...              Ninjutsu  
4  The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...              Taijutsu

In [8]:
df['jutsu_type_simplified'].value_counts()

Ninjutsu    2255
Taijutsu     397
Genjutsu     101
Name: jutsu_type_simplified, dtype: int64

In [9]:
#this data above is a skewed dataset since ninjutsu is far greater than the other two
#ill handle this later

In [10]:
df['text'] = df['jutsu_name'] + ". " + df['jutsu_description']
df['jutsu'] = df['jutsu_type_simplified']
df = df[['text', 'jutsu']]
df = df.dropna()

In [11]:
df.head()

text     jutsu
0  Adamantine Sealing Chains: Spiral Formation. K...  Ninjutsu
1  Adamantine Power: Acala. Hashirama kicks the o...  Ninjutsu
2  Adamantine Prison Wall. After using Transforma...  Ninjutsu
3  Adamantine Seal: Monkey Yang Suppression. Afte...  Ninjutsu
4  Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...  Taijutsu

In [12]:
#some data might have html tags hidden inside since it was scraped from the internet
#in our data it is not the case
#but to make this more generalised ill add a cleaner function

In [20]:
from bs4 import BeautifulSoup
class Cleaner():
    def __init__(self):
        pass 
    
    def put_line_breaks(self, text):
        return text.replace("<\p>", "<\p>\n")
    
    def remove_html_tags(self, text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text

    def clean(self, text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text
        

In [21]:
text_column_name = 'text'
label_column_name = 'jutsu'

In [22]:
# Clean Text
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

In [25]:
df.head(2)

text     jutsu  \
0  Adamantine Sealing Chains: Spiral Formation. K...  Ninjutsu   
1  Adamantine Power: Acala. Hashirama kicks the o...  Ninjutsu   

                                        text_cleaned  
0  Adamantine Sealing Chains: Spiral Formation. K...  
1  Adamantine Power: Acala. Hashirama kicks the o...

In [29]:
#neural networks need numbers for the output
#so we are just using label encoders here for jutsus
# Encode Labels 
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())

LabelEncoder()

In [30]:
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}
label_dict

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [31]:
df['label'] = le.transform(df[label_column_name].tolist())

In [32]:
df.head()

text     jutsu  \
0  Adamantine Sealing Chains: Spiral Formation. K...  Ninjutsu   
1  Adamantine Power: Acala. Hashirama kicks the o...  Ninjutsu   
2  Adamantine Prison Wall. After using Transforma...  Ninjutsu   
3  Adamantine Seal: Monkey Yang Suppression. Afte...  Ninjutsu   
4  Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...  Taijutsu   

                                        text_cleaned  label  
0  Adamantine Sealing Chains: Spiral Formation. K...      1  
1  Adamantine Power: Acala. Hashirama kicks the o...      1  
2  Adamantine Prison Wall. After using Transforma...      1  
3  Adamantine Seal: Monkey Yang Suppression. Afte...      1  
4  Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...      2

In [34]:
test_size = 0.2
df_train, df_test = train_test_split(df, test_size=test_size, random_state=42, stratify=df['label'])
#stratify tell train test split that for each of classes 80 percent should in training and 20 in test

In [36]:
df_train['jutsu'].value_counts()

Ninjutsu    1804
Taijutsu     317
Genjutsu      81
Name: jutsu, dtype: int64

In [37]:
model_name = "distilbert/distilbert-base-uncased"

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/shiven/anaconda3/envs/gradio_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [42]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples['text_cleaned'], truncation=True)

In [45]:
#convert pandas to a hugging face dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

#tokenize the dataset
tokenized_train = train_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True) #batched to prevent it from 
                                                  #running into memory issues
tokenized_test = test_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)

Map: 100%|██████████| 551/551 [00:00<00:00, 6603.62 examples/s]
